In [ ]:
import psycopg2
import pandas as pd

# --- Connect to PostgreSQL MIMIC-IV database ---
# Database connection parameters
DB_NAME = "mimiciv"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_USER = "Enter database username: "
DB_PASSWORD = "Enter database password: "

# Attempt to connect to the database
try:
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    print("Connected to the database.")
except Exception as e:
    print("Connection failed:", e)

# --- Query 1: Unique stay_ids from vitalsign table ---
query_vitals_stays = """
SELECT DISTINCT stay_id FROM mimiciv_derived.vitalsign;
"""
df_vitals_stays = pd.read_sql(query_vitals_stays, conn)
print(f"Unique stays from vitalsign: {len(df_vitals_stays)}")

# --- Query 2: Unique stay_ids from labevents (joined with icustays) ---
query_labs_stays = """
SELECT DISTINCT icu.stay_id
FROM mimiciv_hosp.labevents le
JOIN mimiciv_icu.icustays icu
  ON le.subject_id = icu.subject_id
  AND le.hadm_id = icu.hadm_id
WHERE le.valuenum IS NOT NULL;
"""
df_labs_stays = pd.read_sql(query_labs_stays, conn)
print(f"Unique stays from labevents: {len(df_labs_stays)}")

# --- Save results to CSV ---
df_vitals_stays.to_csv("stays_vitals.csv", index=False)
df_labs_stays.to_csv("stays_labs.csv", index=False)
print("Stay lists saved to stays_vitals.csv and stays_labs.csv.")

conn.close()

In [ ]:
import psycopg2
import pandas as pd
from sklearn.model_selection import train_test_split

# --- Connect to PostgreSQL database ---
# Database connection parameters
DB_NAME = "mimiciv"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_USER = "Enter database username: "
DB_PASSWORD = "Enter database password: "

# Attempt to connect to the database
try:
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    print("Connected to the database.")
except Exception as e:
    print("Connection failed:", e)

# --- Step 1: Map subject_id ↔ stay_id from icustays ---
query_map = "SELECT subject_id, stay_id FROM mimiciv_icu.icustays;"
df_map = pd.read_sql(query_map, conn)
df_map.to_csv("subject_stay_map.csv", index=False)
print("Mapping saved to subject_stay_map.csv")

conn.close()

# --- Step 2: Load previously saved stay_id sets ---
df_vitals_stays = pd.read_csv("stays_vitals.csv")
df_labs_stays = pd.read_csv("stays_labs.csv")

stays_labs = df_labs_stays['stay_id'].unique()
stays_vitals = df_vitals_stays['stay_id'].unique()
stays_common = set(stays_labs).intersection(stays_vitals)

# --- Step 3: Filter subject_id with common stays only ---
df_valid = df_map[df_map['stay_id'].isin(stays_common)].copy()

# --- Step 4: Split by subject_id (to prevent leakage) ---
unique_subjects = df_valid['subject_id'].unique()
train_subj, test_subj = train_test_split(unique_subjects, test_size=0.15, random_state=42)
train_subj, val_subj = train_test_split(train_subj, test_size=0.1765, random_state=42)  # ≈15% val

# --- Step 5: Retrieve stay_ids from subject splits ---
def stays_from(subjects):
    return df_valid[df_valid['subject_id'].isin(subjects)]['stay_id'].unique()

split_df = pd.concat([
    pd.DataFrame({'stay_id': stays_from(train_subj), 'split': 'train'}),
    pd.DataFrame({'stay_id': stays_from(val_subj), 'split': 'val'}),
    pd.DataFrame({'stay_id': stays_from(test_subj), 'split': 'test'}),
])

split_df.to_csv("split_all.tsv", sep='\t', index=False)
print("Split saved to split_all.tsv")